<a href="https://colab.research.google.com/github/diksha139/movieRecommedationApp/blob/main/ContentBasedFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/procodingclass/kaggle-movie-data-.git

Cloning into 'kaggle-movie-data-'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 7 (delta 0), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.


In [3]:
import pandas as pd
import numpy as np
movies_df = pd.read_csv('/content/kaggle-movie-data-/tmdb_5000_movies.csv')
credits_df = pd.read_csv('/content/kaggle-movie-data-/tmdb_5000_credits.csv')

In [4]:
credits_df.rename(columns = {'movie_id' : 'id'} , inplace = True)

In [5]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4799 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [6]:
credits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      4803 non-null   int64 
 1   title   4803 non-null   object
 2   cast    4803 non-null   object
 3   crew    4802 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [7]:
common_df = pd.merge(movies_df , credits_df , on = 'id')

In [8]:
common_df.drop(['homepage', 'title_x', 'title_y', 'production_companies'] , axis = 1 , inplace = True)
common_df.dropna(inplace = True)
common_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3958 entries, 0 to 4801
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                3958 non-null   int64  
 1   genres                3958 non-null   object 
 2   id                    3958 non-null   int64  
 3   keywords              3958 non-null   object 
 4   original_language     3958 non-null   object 
 5   original_title        3958 non-null   object 
 6   overview              3958 non-null   object 
 7   popularity            3958 non-null   float64
 8   production_countries  3958 non-null   object 
 9   release_date          3958 non-null   object 
 10  revenue               3958 non-null   int64  
 11  runtime               3958 non-null   float64
 12  spoken_languages      3958 non-null   object 
 13  status                3958 non-null   object 
 14  tagline               3958 non-null   object 
 15  vote_average         

In [9]:
# calculating weighted_rating
R = common_df['vote_average']
v = common_df['vote_count']
C = common_df['vote_average'].mean()
print(C)
m = common_df['vote_count'].quantile(0.9)
print(m)
common_df['weighted_rating'] = (R*v + C*m) / (v + m)

6.181859525012634
2101.2000000000007


In [10]:
common_df[['cast','genres','keywords']].head(1)



,cast,genres,keywords
0,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."


In [20]:
common_df.loc[0].at['keywords']

['cultureclash',
 'future',
 'spacewar',
 'spacecolony',
 'society',
 'spacetravel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alienplanet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'loveaffair',
 'antiwar',
 'powerrelations',
 'mindandsoul',
 '3d']

In [12]:
from ast import literal_eval

features = ['cast' , 'crew', 'keywords', 'genres']
for feature in features:
  common_df[feature] = common_df[feature].apply(literal_eval)

In [13]:
def get_director(crew):
  for crew_member in crew:
    if crew_member['job']  ==  'Director':
      return crew_member['name']
    
  return np.nan

common_df['director'] = common_df['crew'].apply(get_director)

In [14]:
def get_name_list(column_value):
  names_list = []
  if isinstance(column_value , list):
    for element in column_value:
      names_list.append(element['name'])

  return names_list

features = ['cast' , 'keywords' , 'genres']
for feature in features:
  common_df[feature] = common_df[feature].apply(get_name_list)

In [15]:
def clean_data(column_value):
  modified_list = []
  modified_string = ""
  if isinstance(column_value , list):
    for element in column_value:
      modified_string = element.replace(" " , "")
      modified_list.append(modified_string.lower())

    return modified_list

  elif isinstance(column_value , str):
    modified_string = column_value.replace(" " , "")
    return modified_string.lower()

  else:
    return ''

features = ['cast' , 'keywords' , 'genres' , 'director']
for feature in features:
  common_df[feature] = common_df[feature].apply(clean_data)

In [22]:
l1=['a','b','c']
"$".join(l1)

'a$b$c'

In [16]:
def create_soup(x):
   return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
common_df['soup'] = common_df.apply(create_soup, axis=1)

In [17]:
# resetting index
common_df = common_df.reset_index()
indices = pd.Series(common_df.index, index=common_df['original_title'])

In [18]:
# converting dataframe to csv
common_df.to_csv('movies.csv')

In [23]:
# downloading file
from google.colab import files
files.download('movies.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>